In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/foodseg103/FoodSeg103/category_id.txt
/kaggle/input/foodseg103/FoodSeg103/train_test_recipe1m_id.txt
/kaggle/input/foodseg103/FoodSeg103/Readme.txt
/kaggle/input/foodseg103/FoodSeg103/test_recipe1m_id.txt
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00007111.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005029.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00006166.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00004975.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005750.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005865.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005643.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005039.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005239.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00004669.jpg
/kaggle/input/foodseg103/FoodSeg103/Images/img_dir/test/00005170.jpg
/kaggle/input/foodseg1

In [2]:
"""
DeepLab Fine-Tuning on FoodSeg103 Dataset
With Custom Hand-Coded Attention Modules
التزام كامل بشروط المشروع - كل Attention Modules مكتوبة يدوياً
"""

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50, deeplabv3_resnet101
from PIL import Image
import numpy as np
from tqdm import tqdm
import random

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ============================================================================
# STEP 1: Custom Attention Modules (Hand-Coded من الصفر)
# ============================================================================

class ChannelAttentionModule(nn.Module):
    """
    Channel Attention Module - مكتوب يدوياً
    يركز على "أي قنوات مهمة" في الfeatures
    """
    def __init__(self, channels, reduction_ratio=16):
        super(ChannelAttentionModule, self).__init__()
        self.channels = channels
        self.reduction_ratio = reduction_ratio
        
        # Shared MLP layers
        self.fc1 = nn.Linear(channels, channels // reduction_ratio, bias=False)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(channels // reduction_ratio, channels, bias=False)
        
        # Pools
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        print(f"  → Channel Attention: {channels} channels, reduction={reduction_ratio}")
    
    def forward(self, x):
        batch, channels, _, _ = x.size()
        
        # Average pooling path
        avg_out = self.avg_pool(x).view(batch, channels)
        avg_out = self.fc1(avg_out)
        avg_out = self.relu(avg_out)
        avg_out = self.fc2(avg_out)
        
        # Max pooling path
        max_out = self.max_pool(x).view(batch, channels)
        max_out = self.fc1(max_out)
        max_out = self.relu(max_out)
        max_out = self.fc2(max_out)
        
        # Combine and apply sigmoid
        channel_attention = torch.sigmoid(avg_out + max_out)
        channel_attention = channel_attention.view(batch, channels, 1, 1)
        
        # Apply attention
        return x * channel_attention.expand_as(x)


class SpatialAttentionModule(nn.Module):
    """
    Spatial Attention Module - مكتوب يدوياً
    يركز على "أين المناطق المهمة" في الصورة
    """
    def __init__(self, kernel_size=7):
        super(SpatialAttentionModule, self).__init__()
        self.kernel_size = kernel_size
        
        # Convolutional layer
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv2d(2, 1, kernel_size=kernel_size, 
                             padding=padding, bias=False)
        
        print(f"  → Spatial Attention: kernel_size={kernel_size}")
    
    def forward(self, x):
        # Average pooling across channels
        avg_out = torch.mean(x, dim=1, keepdim=True)
        
        # Max pooling across channels
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        
        # Concatenate
        concat = torch.cat([avg_out, max_out], dim=1)
        
        # Apply convolution and sigmoid
        spatial_attention = torch.sigmoid(self.conv(concat))
        
        # Apply attention
        return x * spatial_attention


class CBAM(nn.Module):
    """
    CBAM: Convolutional Block Attention Module - مكتوب يدوياً بالكامل
    يجمع بين Channel Attention و Spatial Attention
    """
    def __init__(self, channels, reduction_ratio=16, kernel_size=7):
        super(CBAM, self).__init__()
        print(f"→ Building CBAM Module for {channels} channels:")
        
        self.channel_attention = ChannelAttentionModule(channels, reduction_ratio)
        self.spatial_attention = SpatialAttentionModule(kernel_size)
    
    def forward(self, x):
        # Apply channel attention first
        x = self.channel_attention(x)
        # Then apply spatial attention
        x = self.spatial_attention(x)
        return x


class SelfAttention(nn.Module):
    """
    Self-Attention Module - مكتوب يدوياً
    للتركيز على العلاقات بين أجزاء الصورة المختلفة
    """
    def __init__(self, in_channels):
        super(SelfAttention, self).__init__()
        self.in_channels = in_channels
        
        # Query, Key, Value projections
        self.query_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.key_conv = nn.Conv2d(in_channels, in_channels // 8, kernel_size=1)
        self.value_conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        
        # Learnable parameter
        self.gamma = nn.Parameter(torch.zeros(1))
        
        print(f"→ Self-Attention Module: {in_channels} channels")
    
    def forward(self, x):
        batch, C, H, W = x.size()
        
        # Generate Query, Key, Value
        query = self.query_conv(x).view(batch, -1, H * W).permute(0, 2, 1)  # B x (H*W) x C'
        key = self.key_conv(x).view(batch, -1, H * W)  # B x C' x (H*W)
        value = self.value_conv(x).view(batch, -1, H * W)  # B x C x (H*W)
        
        # Attention map
        attention = torch.bmm(query, key)  # B x (H*W) x (H*W)
        attention = F.softmax(attention, dim=-1)
        
        # Apply attention to value
        out = torch.bmm(value, attention.permute(0, 2, 1))  # B x C x (H*W)
        out = out.view(batch, C, H, W)
        
        # Residual connection with learnable weight
        out = self.gamma * out + x
        
        return out


class MultiScaleAttention(nn.Module):
    """
    Multi-Scale Attention Module - مكتوب يدوياً
    يركز على features بمقاسات مختلفة
    """
    def __init__(self, channels):
        super(MultiScaleAttention, self).__init__()
        
        # Different scales
        self.scale1 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True)
        )
        
        self.scale2 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True)
        )
        
        self.scale3 = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=5, padding=2),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True)
        )
        
        # Fusion layer
        self.fusion = nn.Sequential(
            nn.Conv2d(channels * 3, channels, kernel_size=1),
            nn.BatchNorm2d(channels),
            nn.Sigmoid()
        )
        
        print(f"→ Multi-Scale Attention: {channels} channels (scales: 1x1, 3x3, 5x5)")
    
    def forward(self, x):
        s1 = self.scale1(x)
        s2 = self.scale2(x)
        s3 = self.scale3(x)
        
        # Concatenate all scales
        concat = torch.cat([s1, s2, s3], dim=1)
        
        # Generate attention weights
        attention = self.fusion(concat)
        
        # Apply attention
        return x * attention


# ============================================================================
# STEP 2: Enhanced Dataset with Data Augmentation
# ============================================================================

class FoodSeg103Dataset(Dataset):
    """Custom Dataset for FoodSeg103 with data augmentation"""
    
    def __init__(self, root_dir, split='train', img_size=384, augment=True):
        self.root_dir = root_dir
        self.split = split
        self.img_size = img_size
        self.augment = augment and (split == 'train')
        
        # Normalization
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
        
        # Find paths
        self.img_dir, self.ann_dir, self.id_file = self._find_paths(root_dir, split)
        
        # Load image IDs
        if self.id_file and os.path.exists(self.id_file):
            print(f"Loading {split} image IDs from: {self.id_file}")
            with open(self.id_file, 'r') as f:
                self.image_ids = [line.strip() for line in f.readlines()]
            print(f"Loaded {len(self.image_ids)} {split} images")
        else:
            print(f"Scanning directory: {self.img_dir}")
            if os.path.exists(self.img_dir):
                self.image_ids = [f.replace('.jpg', '').replace('.png', '') 
                                 for f in os.listdir(self.img_dir) 
                                 if f.endswith(('.jpg', '.png'))]
                print(f"Found {len(self.image_ids)} {split} images")
            else:
                raise FileNotFoundError(f"Image directory not found: {self.img_dir}")
    
    def _find_paths(self, root_dir, split):
        patterns = [
            (os.path.join(root_dir, 'FoodSeg103', 'Images', 'img_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'ImageSets', f'{split}.txt')),
            (os.path.join(root_dir, 'Images', 'img_dir', split),
             os.path.join(root_dir, 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'ImageSets', f'{split}.txt')),
        ]
        
        for img_dir, ann_dir, id_file in patterns:
            if os.path.exists(img_dir) and os.path.exists(ann_dir):
                print(f"✓ Found: {img_dir}")
                print(f"✓ Found: {ann_dir}")
                if os.path.exists(id_file):
                    print(f"✓ Found: {id_file}")
                return img_dir, ann_dir, id_file
        
        raise FileNotFoundError(f"Could not find {split} data in {root_dir}")
    
    def __len__(self):
        return len(self.image_ids)
    
    def _apply_augmentation(self, image, mask):
        # Horizontal flip
        if random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)
        
        # Vertical flip
        if random.random() > 0.3:
            image = transforms.functional.vflip(image)
            mask = transforms.functional.vflip(mask)
        
        # Random rotation
        if random.random() > 0.5:
            angle = random.choice([90, 180, 270])
            image = transforms.functional.rotate(image, angle)
            mask = transforms.functional.rotate(mask, angle, 
                                               interpolation=transforms.InterpolationMode.NEAREST)
        
        # Color jitter
        if random.random() > 0.5:
            color_jitter = transforms.ColorJitter(
                brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1
            )
            image = color_jitter(image)
        
        return image, mask
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_id = img_id.replace('.jpg', '').replace('.png', '')
        
        # Find image
        img_path = None
        for ext in ['.jpg', '.png', '.jpeg']:
            temp_path = os.path.join(self.img_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                img_path = temp_path
                break
        
        if img_path is None:
            raise FileNotFoundError(f"Image not found: {img_id}")
        
        # Find mask
        mask_path = None
        for ext in ['.png', '.jpg']:
            temp_path = os.path.join(self.ann_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                mask_path = temp_path
                break
        
        if mask_path is None:
            raise FileNotFoundError(f"Mask not found: {img_id}")
        
        # Load
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path)
        
        # Resize
        image = transforms.functional.resize(image, (self.img_size, self.img_size))
        mask = transforms.functional.resize(mask, (self.img_size, self.img_size), 
                                           interpolation=transforms.InterpolationMode.NEAREST)
        
        # Augmentation
        if self.augment:
            image, mask = self._apply_augmentation(image, mask)
        
        # To tensor
        image = transforms.functional.to_tensor(image)
        image = self.normalize(image)
        
        mask = np.array(mask, dtype=np.int64)
        mask = torch.from_numpy(mask).long()
        
        return image, mask


# ============================================================================
# STEP 3: DeepLab with Custom Attention Modules
# ============================================================================

class DeepLabWithCustomAttention(nn.Module):
    """
    DeepLab with Custom Hand-Coded Attention Modules
    جميع الAttention Modules مكتوبة يدوياً من الصفر
    """
    def __init__(self, num_classes=104, backbone='resnet50', pretrained=True,
                 attention_type='cbam'):
        """
        Args:
            num_classes: عدد الفئات
            backbone: 'resnet50' or 'resnet101' (pretrained مسموح)
            pretrained: استخدام backbone pretrained (مسموح)
            attention_type: نوع الAttention ('cbam', 'self', 'multiscale', 'all')
        """
        super(DeepLabWithCustomAttention, self).__init__()
        
        print(f"\n{'='*70}")
        print(f"Building Custom DeepLab with Hand-Coded Attention")
        print(f"{'='*70}")
        print(f"Backbone: {backbone} (pretrained={pretrained})")
        print(f"Attention Type: {attention_type}")
        print(f"Number of Classes: {num_classes}")
        
        # Load pretrained backbone (مسموح حسب التعليمات)
        if backbone == 'resnet50':
            base_model = deeplabv3_resnet50(pretrained=pretrained)
        elif backbone == 'resnet101':
            base_model = deeplabv3_resnet101(pretrained=pretrained)
        else:
            raise ValueError(f"Unknown backbone: {backbone}")
        
        # Extract components
        self.backbone = base_model.backbone
        self.classifier = base_model.classifier
        self.aux_classifier = base_model.aux_classifier
        
        # Replace final layers
        self.classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)
        self.aux_classifier[4] = nn.Conv2d(256, num_classes, kernel_size=1)
        
        # Add Custom Attention Modules (مكتوبة يدوياً)
        print(f"\nAdding Custom Attention Modules:")
        print(f"{'-'*70}")
        
        self.attention_type = attention_type
        
        if attention_type == 'cbam':
            self.attention1 = CBAM(256, reduction_ratio=16, kernel_size=7)
            
        elif attention_type == 'self':
            self.attention1 = SelfAttention(256)
            
        elif attention_type == 'multiscale':
            self.attention1 = MultiScaleAttention(256)
            
        elif attention_type == 'all':
            # استخدام كل الAttention Modules معاً
            self.cbam = CBAM(256, reduction_ratio=16, kernel_size=7)
            self.self_attn = SelfAttention(256)
            self.multiscale = MultiScaleAttention(256)
            
        else:
            raise ValueError(f"Unknown attention type: {attention_type}")
        
        print(f"{'-'*70}")
        print(f"✓ Model with Custom Attention Built Successfully!")
        print(f"{'='*70}\n")
    
    def forward(self, x):
        input_shape = x.shape[-2:]
        
        # Backbone features
        features = self.backbone(x)
        
        # ASPP module
        x_aspp = self.classifier[0](features['out'])
        x_aspp = self.classifier[1](x_aspp)
        x_aspp = self.classifier[2](x_aspp)
        x_aspp = self.classifier[3](x_aspp)
        
        # Apply Custom Attention Modules
        if self.attention_type == 'all':
            # Apply all attention modules sequentially
            x_aspp = self.cbam(x_aspp)
            x_aspp = self.self_attn(x_aspp)
            x_aspp = self.multiscale(x_aspp)
        else:
            # Apply single attention module
            x_aspp = self.attention1(x_aspp)
        
        # Final classifier
        x_main = self.classifier[4](x_aspp)
        x_main = F.interpolate(x_main, size=input_shape, 
                              mode='bilinear', align_corners=False)
        
        result = {'out': x_main}
        
        # Auxiliary classifier
        if self.training and 'aux' in features:
            x_aux = self.aux_classifier(features['aux'])
            x_aux = F.interpolate(x_aux, size=input_shape, 
                                 mode='bilinear', align_corners=False)
            result['aux'] = x_aux
        
        return result


# ============================================================================
# STEP 4: Training Functions
# ============================================================================

def calculate_metrics(pred, target, num_classes=104):
    """Calculate pixel accuracy and mean IoU"""
    pred = pred.cpu().numpy()
    target = target.cpu().numpy()
    
    # Pixel accuracy
    correct = (pred == target).sum()
    total = target.size
    pixel_acc = correct / total
    
    # Mean IoU
    iou_list = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        
        intersection = (pred_cls & target_cls).sum()
        union = (pred_cls | target_cls).sum()
        
        if union > 0:
            iou_list.append(intersection / union)
    
    mean_iou = np.mean(iou_list) if iou_list else 0
    
    return pixel_acc, mean_iou


def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch, scaler):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    running_pixel_acc = 0.0
    running_miou = 0.0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")
    
    for i, (images, masks) in enumerate(pbar):
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs['out'], masks)
            
            if 'aux' in outputs:
                aux_loss = criterion(outputs['aux'], masks)
                loss = loss + 0.4 * aux_loss
        
        # Backward
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Metrics
        with torch.no_grad():
            preds = outputs['out'].argmax(dim=1)
            pixel_acc, miou = calculate_metrics(preds, masks)
        
        running_loss += loss.item()
        running_pixel_acc += pixel_acc
        running_miou += miou
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{pixel_acc:.4f}',
            'mIoU': f'{miou:.4f}'
        })
    
    epoch_loss = running_loss / len(dataloader)
    epoch_pixel_acc = running_pixel_acc / len(dataloader)
    epoch_miou = running_miou / len(dataloader)
    
    return epoch_loss, epoch_pixel_acc, epoch_miou


@torch.no_grad()
def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    running_pixel_acc = 0.0
    running_miou = 0.0
    
    pbar = tqdm(dataloader, desc="Validation")
    
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        outputs = model(images)
        loss = criterion(outputs['out'], masks)
        
        preds = outputs['out'].argmax(dim=1)
        pixel_acc, miou = calculate_metrics(preds, masks)
        
        running_loss += loss.item()
        running_pixel_acc += pixel_acc
        running_miou += miou
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{pixel_acc:.4f}',
            'mIoU': f'{miou:.4f}'
        })
    
    val_loss = running_loss / len(dataloader)
    val_pixel_acc = running_pixel_acc / len(dataloader)
    val_miou = running_miou / len(dataloader)
    
    return val_loss, val_pixel_acc, val_miou


def get_parameter_groups(model, backbone_lr, head_lr):
    """Create parameter groups with different learning rates"""
    backbone_params = []
    head_params = []
    attention_params = []
    
    for name, param in model.named_parameters():
        if 'attention' in name or 'cbam' in name or 'self_attn' in name or 'multiscale' in name:
            attention_params.append(param)
        elif 'classifier' in name or 'aux_classifier' in name:
            head_params.append(param)
        else:
            backbone_params.append(param)
    
    param_groups = [
        {'params': backbone_params, 'lr': backbone_lr},
        {'params': head_params, 'lr': head_lr},
        {'params': attention_params, 'lr': head_lr}
    ]
    
    print(f"Backbone params: {len(backbone_params)} (lr={backbone_lr})")
    print(f"Head params: {len(head_params)} (lr={head_lr})")
    print(f"Attention params: {len(attention_params)} (lr={head_lr})")
    
    return param_groups


# ============================================================================
# STEP 5: Main Training Function
# ============================================================================

def train_deeplab_foodseg103(
    root_dir, 
    num_epochs=25,
    batch_size=8,
    backbone_lr=1e-4,
    head_lr=1e-3,
    img_size=384,
    backbone='resnet50',
    attention_type='cbam',
    warmup_epochs=3
):
    """Main training function"""
    
    set_seed(42)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"\nUsing device: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Load datasets
    print(f"\n{'='*70}")
    print("Loading Datasets")
    print(f"{'='*70}")
    
    train_dataset = FoodSeg103Dataset(root_dir, 'train', img_size, augment=True)
    test_dataset = FoodSeg103Dataset(root_dir, 'test', img_size, augment=False)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,
                             num_workers=4, pin_memory=True, persistent_workers=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                            num_workers=4, pin_memory=True, persistent_workers=True)
    
    print(f"Train batches: {len(train_loader)} | Test batches: {len(test_loader)}")
    
    # Create model with custom attention
    model = DeepLabWithCustomAttention(
        num_classes=104,
        backbone=backbone,
        pretrained=True,
        attention_type=attention_type
    )
    model = model.to(device)
    
    # Optimizer
    param_groups = get_parameter_groups(model, backbone_lr, head_lr)
    optimizer = optim.AdamW(param_groups, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)
    
    # Loss
    criterion = nn.CrossEntropyLoss(ignore_index=255, label_smoothing=0.1)
    scaler = torch.cuda.amp.GradScaler()
    
    # Training history
    history = {
        'train_loss': [], 'train_acc': [], 'train_miou': [],
        'val_loss': [], 'val_acc': [], 'val_miou': []
    }
    
    # Training loop
    print(f"\n{'='*70}")
    print("Training Started")
    print(f"{'='*70}\n")
    
    best_miou = 0.0
    
    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}")
        print("-" * 70)
        
        # Train
        train_loss, train_acc, train_miou = train_one_epoch(
            model, train_loader, criterion, optimizer, device, epoch, scaler
        )
        
        # Validate
        val_loss, val_acc, val_miou = validate(model, test_loader, criterion, device)
        
        scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_miou'].append(train_miou)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_miou'].append(val_miou)
        
        # Print summary
        print(f"\n{'='*70}")
        print(f"Epoch {epoch} Summary:")
        print(f"Train → Loss: {train_loss:.4f} | Acc: {train_acc:.4f} | mIoU: {train_miou:.4f}")
        print(f"Val   → Loss: {val_loss:.4f} | Acc: {val_acc:.4f} | mIoU: {val_miou:.4f}")
        print(f"{'='*70}")
        
        # Save best
        if val_miou > best_miou:
            best_miou = val_miou
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_miou': best_miou,
                'history': history
            }, 'best_deeplab_custom_attention.pth')
            print(f"✓ Saved best model | mIoU: {best_miou:.4f}")
    
    print(f"\n{'='*70}")
    print("Training Complete!")
    print(f"Best validation mIoU: {best_miou:.4f}")
    print(f"{'='*70}\n")
    
    return model, history


# ============================================================================
# STEP 6: Main Execution
# ============================================================================

if __name__ == "__main__":
    ROOT_DIR = "/kaggle/input/foodseg103"
    
    # Configuration
    config = {
        'num_epochs': 25,
        'batch_size': 8,

SyntaxError: incomplete input (1964826909.py, line 745)

In [ ]:
"""
DeepLab Fine-Tuning on FoodSeg103 Dataset
Optimized for speed and accuracy with proper fine-tuning
"""

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet50
from PIL import Image
import numpy as np
from tqdm import tqdm
import random

# Set seeds for reproducibility
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# ============================================================================
# STEP 1: Enhanced Dataset with Data Augmentation
# ============================================================================

class FoodSeg103Dataset(Dataset):
    """Custom Dataset for FoodSeg103 with data augmentation"""
    
    def __init__(self, root_dir, split='train', img_size=512, augment=True):
        """
        Args:
            root_dir: Root directory of FoodSeg103
            split: 'train' or 'test'
            img_size: Target size for images and masks
            augment: Whether to apply data augmentation
        """
        self.root_dir = root_dir
        self.split = split
        self.img_size = img_size
        self.augment = augment and (split == 'train')
        
        # Normalization (ImageNet stats)
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
        
        # Try to auto-detect directory structure
        self.img_dir, self.ann_dir, self.id_file = self._find_paths(root_dir, split)
        
        # Load image IDs
        if self.id_file and os.path.exists(self.id_file):
            print(f"Loading {split} image IDs from: {self.id_file}")
            with open(self.id_file, 'r') as f:
                self.image_ids = [line.strip() for line in f.readlines()]
            print(f"Loaded {len(self.image_ids)} {split} images")
        else:
            print(f"No ID file found, scanning directory: {self.img_dir}")
            if os.path.exists(self.img_dir):
                self.image_ids = [f.replace('.jpg', '').replace('.png', '') 
                                 for f in os.listdir(self.img_dir) 
                                 if f.endswith(('.jpg', '.png'))]
                print(f"Found {len(self.image_ids)} {split} images")
            else:
                raise FileNotFoundError(f"Image directory not found: {self.img_dir}")
    
    def _find_paths(self, root_dir, split):
        """Auto-detect the directory structure"""
        patterns = [
            (os.path.join(root_dir, 'FoodSeg103', 'Images', 'img_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'ImageSets', f'{split}.txt')),
            (os.path.join(root_dir, 'Images', 'img_dir', split),
             os.path.join(root_dir, 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'ImageSets', f'{split}.txt')),
        ]
        
        for img_dir, ann_dir, id_file in patterns:
            if os.path.exists(img_dir) and os.path.exists(ann_dir):
                print(f"✓ Found image directory: {img_dir}")
                print(f"✓ Found annotation directory: {ann_dir}")
                if os.path.exists(id_file):
                    print(f"✓ Found ID file: {id_file}")
                else:
                    print(f"⚠ ID file not found: {id_file} (will scan directory)")
                return img_dir, ann_dir, id_file
        
        raise FileNotFoundError(
            f"Could not find {split} images and annotations in {root_dir}"
        )
    
    def __len__(self):
        return len(self.image_ids)
    
    def _apply_augmentation(self, image, mask):
        """Apply synchronized augmentations to image and mask"""
        # Random horizontal flip
        if random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)
        
        return image, mask
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx]
        img_id = img_id.replace('.jpg', '').replace('.png', '')
        
        # Find image path
        img_path = None
        for ext in ['.jpg', '.png', '.jpeg']:
            temp_path = os.path.join(self.img_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                img_path = temp_path
                break
        
        if img_path is None:
            raise FileNotFoundError(f"Image not found: {img_id}")
        
        # Find mask path
        mask_path = None
        for ext in ['.png', '.jpg']:
            temp_path = os.path.join(self.ann_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                mask_path = temp_path
                break
        
        if mask_path is None:
            raise FileNotFoundError(f"Mask not found: {img_id}")
        
        # Load image and mask
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path)
        
        # Resize
        image = transforms.functional.resize(image, (self.img_size, self.img_size))
        mask = transforms.functional.resize(mask, (self.img_size, self.img_size), 
                                           interpolation=transforms.InterpolationMode.NEAREST)
        
        # Apply augmentation if training
        if self.augment:
            image, mask = self._apply_augmentation(image, mask)
        
        # Convert to tensor
        image = transforms.functional.to_tensor(image)
        image = self.normalize(image)
        
        mask = np.array(mask, dtype=np.int64)
        mask = torch.from_numpy(mask).long()
        
        return image, mask

# ============================================================================
# STEP 2: Attention Module - CBAM (Convolutional Block Attention Module)
# ============================================================================

class ChannelAttention(nn.Module):
    """Channel Attention Module"""
    def __init__(self, channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    """Spatial Attention Module"""
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=kernel_size//2, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        out = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(out)
        return self.sigmoid(out)

class CBAM(nn.Module):
    """CBAM: Convolutional Block Attention Module"""
    def __init__(self, channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)
    
    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# ============================================================================
# STEP 3: DeepLab with Attention Module
# ============================================================================

class ASPP(nn.Module):
    """Atrous Spatial Pyramid Pooling"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 1, bias=False)
        self.conv6 = nn.Conv2d(in_channels, out_channels, 3, padding=6, dilation=6, bias=False)
        self.conv12 = nn.Conv2d(in_channels, out_channels, 3, padding=12, dilation=12, bias=False)
        self.conv18 = nn.Conv2d(in_channels, out_channels, 3, padding=18, dilation=18, bias=False)

        self.project = nn.Sequential(
            nn.Conv2d(out_channels * 4, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = self.conv6(x)
        x3 = self.conv12(x)
        x4 = self.conv18(x)
        x = torch.cat([x1, x2, x3, x4], dim=1)
        return self.project(x)

class DeepLabWithAttention(nn.Module):
    """DeepLab with CBAM Attention"""
    def __init__(self, num_classes=104, pretrained=True, use_attention=True):
        super().__init__()

        backbone = resnet50(pretrained=pretrained)

        self.layer0 = nn.Sequential(
            backbone.conv1,
            backbone.bn1,
            backbone.relu,
            backbone.maxpool
        )
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4

        self.aspp = ASPP(2048, 256)
        
        # Optional attention module
        self.use_attention = use_attention
        if use_attention:
            self.attention = CBAM(256)

        self.decoder = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, num_classes, 1)
        )

    def forward(self, x):
        input_size = x.shape[-2:]

        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.aspp(x)
        
        if self.use_attention:
            x = self.attention(x)
            
        x = self.decoder(x)

        x = nn.functional.interpolate(
            x, size=input_size, mode='bilinear', align_corners=False
        )

        return {'out': x}

# ============================================================================
# STEP 4: Model Setup
# ============================================================================

def get_parameter_groups(model, backbone_lr, classifier_lr):
    """
    Create parameter groups with different learning rates
    - Backbone (pre-trained): lower learning rate
    - Classifier & Attention (new): higher learning rate
    """
    backbone_params = []
    head_params = []
    
    for name, param in model.named_parameters():
        if 'layer0' in name or 'layer1' in name or 'layer2' in name or \
           'layer3' in name or 'layer4' in name:
            backbone_params.append(param)
        else:
            head_params.append(param)
    
    param_groups = [
        {'params': backbone_params, 'lr': backbone_lr},
        {'params': head_params, 'lr': classifier_lr}
    ]
    
    print(f"Backbone parameters: {len(backbone_params)} (lr={backbone_lr})")
    print(f"Head parameters: {len(head_params)} (lr={classifier_lr})")
    
    return param_groups

# ============================================================================
# STEP 5: Training Functions
# ============================================================================

def calculate_metrics(pred, target, num_classes=104):
    """Calculate pixel accuracy and mean IoU"""
    pred = pred.cpu().numpy()
    target = target.cpu().numpy()
    
    # Pixel accuracy
    correct = (pred == target).sum()
    total = target.size
    pixel_acc = correct / total
    
    # Mean IoU
    iou_list = []
    for cls in range(num_classes):
        pred_cls = (pred == cls)
        target_cls = (target == cls)
        
        intersection = (pred_cls & target_cls).sum()
        union = (pred_cls | target_cls).sum()
        
        if union > 0:
            iou_list.append(intersection / union)
    
    mean_iou = np.mean(iou_list) if iou_list else 0
    
    return pixel_acc, mean_iou

def train_one_epoch(model, dataloader, criterion, optimizer, device, epoch, scaler):
    """Train for one epoch with mixed precision"""
    model.train()
    running_loss = 0.0
    running_pixel_acc = 0.0
    running_miou = 0.0
    
    pbar = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")
    
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        optimizer.zero_grad()
        
        # Mixed precision training
        with torch.cuda.amp.autocast():
            outputs = model(images)
            loss = criterion(outputs['out'], masks)
        
        # Backward pass with gradient scaling
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        # Metrics
        with torch.no_grad():
            preds = outputs['out'].argmax(dim=1)
            pixel_acc, miou = calculate_metrics(preds, masks)
        
        running_loss += loss.item()
        running_pixel_acc += pixel_acc
        running_miou += miou
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{pixel_acc:.4f}',
            'mIoU': f'{miou:.4f}'
        })
    
    epoch_loss = running_loss / len(dataloader)
    epoch_pixel_acc = running_pixel_acc / len(dataloader)
    epoch_miou = running_miou / len(dataloader)
    
    return epoch_loss, epoch_pixel_acc, epoch_miou

@torch.no_grad()
def validate(model, dataloader, criterion, device):
    """Validate the model"""
    model.eval()
    running_loss = 0.0
    running_pixel_acc = 0.0
    running_miou = 0.0
    
    pbar = tqdm(dataloader, desc="Validation")
    
    for images, masks in pbar:
        images = images.to(device)
        masks = masks.to(device)
        
        outputs = model(images)
        loss = criterion(outputs['out'], masks)
        
        preds = outputs['out'].argmax(dim=1)
        pixel_acc, miou = calculate_metrics(preds, masks)
        
        running_loss += loss.item()
        running_pixel_acc += pixel_acc
        running_miou += miou
        
        pbar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{pixel_acc:.4f}',
            'mIoU': f'{miou:.4f}'
        })
    
    val_loss = running_loss / len(dataloader)
    val_pixel_acc = running_pixel_acc / len(dataloader)
    val_miou = running_miou / len(dataloader)
    
    return val_loss, val_pixel_acc, val_miou

# ============================================================================
# STEP 6: Main Training Loop
# ============================================================================

def train_deeplab_foodseg103(
    root_dir, 
    num_epochs=20, 
    batch_size=16,
    backbone_lr=2e-4,
    classifier_lr=2e-3,
    img_size=256,
    warmup_epochs=2,
    use_attention=False
):
    """Main training function"""
    
    set_seed(42)
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
    
    # Load datasets
    print("\n" + "="*70)
    print("Loading Datasets")
    print("="*70)
    
    train_dataset = FoodSeg103Dataset(root_dir, split='train', img_size=img_size, augment=True)
    test_dataset = FoodSeg103Dataset(root_dir, split='test', img_size=img_size, augment=False)
    
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True, 
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    test_loader = DataLoader(
        test_dataset, 
        batch_size=batch_size * 2,
        shuffle=False, 
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
        prefetch_factor=2
    )
    
    print(f"Train batches: {len(train_loader)} | Test batches: {len(test_loader)}")
    
    # Create model
    print("\n" + "="*70)
    print("Initializing Model")
    print("="*70)
    
    model = DeepLabWithAttention(num_classes=104, pretrained=True, use_attention=use_attention)
    model = model.to(device)
    
    param_groups = get_parameter_groups(model, backbone_lr, classifier_lr)
    optimizer = optim.AdamW(param_groups, weight_decay=0.01)
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2, eta_min=1e-6
    )
    
    criterion = nn.CrossEntropyLoss(ignore_index=255, label_smoothing=0.1)
    scaler = torch.cuda.amp.GradScaler()
    
    history = {
        'train_loss': [], 'train_pixel_acc': [], 'train_miou': [],
        'val_loss': [], 'val_pixel_acc': [], 'val_miou': []
    }
    
    # Training loop
    print("\n" + "="*70)
    print("Training")
    print("="*70)
    
    best_miou = 0.0
    patience = 10
    patience_counter = 0
    
    for epoch in range(1, num_epochs + 1):
        print(f"\nEpoch {epoch}/{num_epochs}")
        print("-" * 70)
        
        # Warmup
        if epoch <= warmup_epochs:
            warmup_factor = epoch / warmup_epochs
            for i, param_group in enumerate(optimizer.param_groups):
                if i == 0:  # backbone
                    param_group['lr'] = backbone_lr * warmup_factor
                else:  # head
                    param_group['lr'] = classifier_lr * warmup_factor
        
        # Train
        train_loss, train_pixel_acc, train_miou = train_one_epoch(
            model, train_loader, criterion, optimizer, device, epoch, scaler
        )
        
        # Validate
        val_loss, val_pixel_acc, val_miou = validate(
            model, test_loader, criterion, device
        )
        
        # Update learning rate
        if epoch > warmup_epochs:
            scheduler.step()
        
        # Save history
        history['train_loss'].append(train_loss)
        history['train_pixel_acc'].append(train_pixel_acc)
        history['train_miou'].append(train_miou)
        history['val_loss'].append(val_loss)
        history['val_pixel_acc'].append(val_pixel_acc)
        history['val_miou'].append(val_miou)
        
        # Print summary
        print(f"\n{'='*70}")
        print(f"Epoch {epoch} Summary:")
        print(f"Train → Loss: {train_loss:.4f} | Acc: {train_pixel_acc:.4f} | mIoU: {train_miou:.4f}")
        print(f"Val   → Loss: {val_loss:.4f} | Acc: {val_pixel_acc:.4f} | mIoU: {val_miou:.4f}")
        print(f"{'='*70}")
        
        # Save best model
        if val_miou > best_miou:
            best_miou = val_miou
            patience_counter = 0
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_miou': best_miou,
                'history': history
            }, 'best_deeplab_foodseg103.pth')
            print(f"✓ Saved best model | mIoU: {best_miou:.4f}")
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= patience:
            print(f"\nEarly stopping triggered after {epoch} epochs")
            break
    
    print("\n" + "="*70)
    print("Training Complete!")
    print("="*70)
    print(f"Best validation mIoU: {best_miou:.4f}")
    
    return model, history

# ============================================================================
# Usage Example
# ============================================================================

if __name__ == "__main__":
    ROOT_DIR = "/kaggle/input/foodseg103"
    
    # Training configuration
    config = {
        'num_epochs': 20,
        'batch_size': 16,
        'backbone_lr': 2e-4,
        'classifier_lr': 2e-3,
        'img_size': 256,
        'warmup_epochs': 2,
        'use_attention': False
    }
    
    print("="*70)
    print("DeepLab Fine-Tuning on FoodSeg103")
    print("="*70)
    
    model, history = train_deeplab_foodseg103(
        root_dir=ROOT_DIR,
        **config
    )
    
    print("\n✓ Training completed successfully!")

In [ ]:
"""
Save & Load Trained Model
"""

import torch
from datetime import datetime

# ============================================================================
# Method 1: Backup Current Model
# ============================================================================

def backup_current_model(checkpoint_path='best_deeplab_foodseg103.pth'):
    """
    Create a backup copy of the model before any modifications
    """
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_path = f'backup_model_{timestamp}_mIoU_{checkpoint["best_miou"]:.4f}.pth'
    
    torch.save(checkpoint, backup_path)
    print(f"✓ Backup saved: {backup_path}")
    print(f"  - Epoch: {checkpoint['epoch']}")
    print(f"  - Best mIoU: {checkpoint['best_miou']:.4f}")
    
    return backup_path

# ============================================================================
# Method 2: Save Experiment Checkpoint
# ============================================================================

def save_experiment_checkpoint(model, optimizer, history, experiment_name, best_miou, epoch):
    """
    Save model with specific experiment name
    
    Examples:
        experiment_name = "baseline_resnet50"
        experiment_name = "with_attention_cbam"
        experiment_name = "larger_batch_size"
    """
    checkpoint_path = f'checkpoint_{experiment_name}_epoch{epoch}.pth'
    
    torch.save({
        'experiment_name': experiment_name,
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'best_miou': best_miou,
        'history': history
    }, checkpoint_path)
    
    print(f"✓ Experiment saved: {checkpoint_path}")
    return checkpoint_path

# ============================================================================
# Method 3: Load Model and Resume Training
# ============================================================================

def load_and_resume_training(checkpoint_path, model, optimizer=None):
    """
    Load saved model to resume training or make modifications
    """
    checkpoint = torch.load(checkpoint_path, weights_only=False)
    
    model.load_state_dict(checkpoint['model'])
    print(f"✓ Model loaded from: {checkpoint_path}")
    
    if optimizer and 'optimizer' in checkpoint:
        optimizer.load_state_dict(checkpoint['optimizer'])
        print(f"✓ Optimizer loaded")
    
    start_epoch = checkpoint.get('epoch', 0) + 1
    best_miou = checkpoint.get('best_miou', 0)
    history = checkpoint.get('history', {})
    
    print(f"  - Starting from epoch: {start_epoch}")
    print(f"  - Best mIoU so far: {best_miou:.4f}")
    
    return model, optimizer, start_epoch, best_miou, history

# ============================================================================
# Method 4: Save Multiple Checkpoints
# ============================================================================

def save_multiple_checkpoints(model, optimizer, history, epoch, val_miou, 
                              save_dir='checkpoints', keep_last_n=3):
    """
    Save multiple checkpoints (e.g., last 3 epochs)
    """
    import os
    os.makedirs(save_dir, exist_ok=True)
    
    checkpoint_path = os.path.join(save_dir, f'checkpoint_epoch{epoch}_miou{val_miou:.4f}.pth')
    torch.save({
        'epoch': epoch,
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'val_miou': val_miou,
        'history': history
    }, checkpoint_path)
    
    checkpoints = sorted([f for f in os.listdir(save_dir) if f.startswith('checkpoint_')])
    if len(checkpoints) > keep_last_n:
        for old_checkpoint in checkpoints[:-keep_last_n]:
            os.remove(os.path.join(save_dir, old_checkpoint))
            print(f"✗ Deleted: {old_checkpoint}")
    
    print(f"✓ Saved: {checkpoint_path}")

# ============================================================================
# Usage Examples
# ============================================================================

if __name__ == "__main__":
    
    # ========== Scenario 1: Backup Before Modifications ==========
    print("\n" + "="*70)
    print("Scenario 1: Backup current model")
    print("="*70)
    
    backup_path = backup_current_model('best_deeplab_foodseg103.pth')
    
    
    # ========== Scenario 2: Save New Experiment ==========
    print("\n" + "="*70)
    print("Scenario 2: Start new experiment keeping old model")
    print("="*70)
    
    """
    model_new = DeepLabWithAttention(104, use_attention=True)
    
    checkpoint = torch.load('best_deeplab_foodseg103.pth')
    model_new.load_state_dict(checkpoint['model'], strict=False)
    
    save_experiment_checkpoint(
        model_new, 
        optimizer, 
        history, 
        experiment_name="with_cbam_attention",
        best_miou=0.75,
        epoch=20
    )
    """
    
    
    # ========== Scenario 3: Resume Training ==========
    print("\n" + "="*70)
    print("Scenario 3: Load model and resume training")
    print("="*70)
    
    """
    model = DeepLabWithAttention(104, use_attention=False)
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)
    
    model, optimizer, start_epoch, best_miou, history = load_and_resume_training(
        'best_deeplab_foodseg103.pth',
        model,
        optimizer
    )
    """
    
    
    # ========== Scenario 4: Compare Different Experiments ==========
    print("\n" + "="*70)
    print("Scenario 4: Save multiple versions for comparison")
    print("="*70)
    
    """
    for epoch in range(start_epoch, num_epochs):
        # ... training code ...
        
        if val_miou > best_miou:
            save_multiple_checkpoints(
                model, optimizer, history, 
                epoch, val_miou,
                save_dir='experiments/baseline',
                keep_last_n=3
            )
    """
    
    print("\n" + "="*70)
    print("✓ Ready! Choose the scenario that fits your needs")
    print("="*70)


# ============================================================================
# Tips & Recommendations
# ============================================================================

"""
Tips:

1. Always backup before modifications:
   - Use backup_current_model() before starting new experiments

2. Use clear file naming:
   - Use descriptive names: "baseline", "with_attention", "larger_lr"
   - Include mIoU in filename to track results

3. Save training history:
   - History contains all metrics for plotting graphs
   - Compare different experiments easily

4. Use Git (optional):
   - Create branches for different experiments

5. Suggested improvements for mIoU:
   ✓ Increase epochs
   ✓ Enable attention (use_attention=True)
   ✓ Try larger img_size (384 or 512)
   ✓ Lower learning rate for fine-tuning
   ✓ Add more data augmentation
   ✓ Use ResNet101 instead of ResNet50
"""

In [ ]:
backup_current_model('best_deeplab_foodseg103.pth')

In [ ]:
# First, backup your current model
backup_current_model('best_deeplab_foodseg103.pth')

# Then run with attention
model, history = train_deeplab_foodseg103(
    root_dir="/kaggle/input/foodseg103",
    num_epochs=25,
    batch_size=16,
    backbone_lr=2e-4,
    classifier_lr=2e-3,
    img_size=256,
    warmup_epochs=2,
    use_attention=True  # ← Just change this!
)

In [ ]:
"""
DeepLab with Custom Attention - SPEED OPTIMIZED
"""

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
import numpy as np
from tqdm import tqdm
import random

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True  # SPEED: Enable cudnn autotuner

# ============================================================================
# LIGHTWEIGHT CUSTOM ATTENTION (Faster versions)
# ============================================================================

class FastChannelAttention(nn.Module):
    """Lightweight channel attention"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)  # Only avg pool for speed
        self.fc = nn.Sequential(
            nn.Conv2d(channels, channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels // reduction, channels, 1, bias=False),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        return x * self.fc(self.pool(x))


class FastSpatialAttention(nn.Module):
    """Lightweight spatial attention"""
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(2, 1, 3, padding=1, bias=False)  # Smaller kernel
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        return x * self.sigmoid(self.conv(torch.cat([avg_out, max_out], dim=1)))


class FastCBAM(nn.Module):
    """Fast CBAM implementation"""
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.ca = FastChannelAttention(channels, reduction)
        self.sa = FastSpatialAttention()
    
    def forward(self, x):
        return self.sa(self.ca(x))


class EfficientSelfAttention(nn.Module):
    """Memory-efficient self-attention with downsampling"""
    def __init__(self, channels, reduction=8, downsample=2):
        super().__init__()
        self.downsample = downsample
        self.q = nn.Conv2d(channels, channels // reduction, 1)
        self.k = nn.Conv2d(channels, channels // reduction, 1)
        self.v = nn.Conv2d(channels, channels, 1)
        self.gamma = nn.Parameter(torch.zeros(1))
        
        if downsample > 1:
            self.down = nn.AvgPool2d(downsample)
    
    def forward(self, x):
        B, C, H, W = x.shape
        
        # Downsample for speed
        if self.downsample > 1:
            x_down = self.down(x)
            h, w = x_down.shape[2:]
        else:
            x_down = x
            h, w = H, W
        
        q = self.q(x).view(B, -1, H * W).permute(0, 2, 1)
        k = self.k(x_down).view(B, -1, h * w)
        v = self.v(x_down).view(B, -1, h * w)
        
        attn = torch.softmax(torch.bmm(q, k) / (k.size(1) ** 0.5), dim=-1)
        out = torch.bmm(v, attn.permute(0, 2, 1)).view(B, C, H, W)
        
        return self.gamma * out + x

# ============================================================================
# SPEED-OPTIMIZED Dataset
# ============================================================================

class FastFoodSeg103Dataset(Dataset):
    def __init__(self, root_dir, split='train', img_size=320, augment=True):
        super().__init__()
        self.split = split
        self.img_size = img_size
        self.augment = augment and (split == 'train')
        
        # Precompute normalization
        self.normalize = transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        )
        
        self.img_dir, self.ann_dir, self.id_file = self._find_paths(root_dir, split)
        
        if self.id_file and os.path.exists(self.id_file):
            with open(self.id_file, 'r') as f:
                self.image_ids = [line.strip() for line in f.readlines()]
        else:
            self.image_ids = [f.replace('.jpg', '').replace('.png', '') 
                             for f in os.listdir(self.img_dir) 
                             if f.endswith(('.jpg', '.png'))]
    
    def _find_paths(self, root_dir, split):
        patterns = [
            (os.path.join(root_dir, 'FoodSeg103', 'Images', 'img_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'FoodSeg103', 'ImageSets', f'{split}.txt')),
            (os.path.join(root_dir, 'Images', 'img_dir', split),
             os.path.join(root_dir, 'Images', 'ann_dir', split),
             os.path.join(root_dir, 'ImageSets', f'{split}.txt')),
        ]
        
        for img_dir, ann_dir, id_file in patterns:
            if os.path.exists(img_dir) and os.path.exists(ann_dir):
                return img_dir, ann_dir, id_file
        
        raise FileNotFoundError(f"Could not find {split} data in {root_dir}")
    
    def __len__(self):
        return len(self.image_ids)
    
    def _fast_augment(self, image, mask):
        """Streamlined augmentation - fewer operations"""
        
        # Horizontal flip (50%)
        if random.random() > 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)
        
        # Scale + crop (60%)
        if random.random() > 0.4:
            scale = random.uniform(0.85, 1.15)
            new_size = int(self.img_size * scale)
            
            image = transforms.functional.resize(image, new_size, antialias=True)
            mask = transforms.functional.resize(mask, new_size, 
                                               interpolation=transforms.InterpolationMode.NEAREST)
            
            if scale > 1.0:
                i = random.randint(0, new_size - self.img_size)
                j = random.randint(0, new_size - self.img_size)
                image = transforms.functional.crop(image, i, j, self.img_size, self.img_size)
                mask = transforms.functional.crop(mask, i, j, self.img_size, self.img_size)
            else:
                pad = self.img_size - new_size
                image = transforms.functional.pad(image, [pad//2, pad//2, pad-pad//2, pad-pad//2])
                mask = transforms.functional.pad(mask, [pad//2, pad//2, pad-pad//2, pad-pad//2], fill=255)
        
        # Color jitter (40%)
        if random.random() > 0.6:
            image = transforms.ColorJitter(0.15, 0.15, 0.15, 0.05)(image)
        
        return image, mask
    
    def __getitem__(self, idx):
        img_id = self.image_ids[idx].replace('.jpg', '').replace('.png', '')
        
        # Find paths dynamically (safe for multiprocessing)
        img_path = None
        for ext in ['.jpg', '.png', '.jpeg']:
            temp_path = os.path.join(self.img_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                img_path = temp_path
                break
        
        mask_path = None
        for ext in ['.png', '.jpg']:
            temp_path = os.path.join(self.ann_dir, f'{img_id}{ext}')
            if os.path.exists(temp_path):
                mask_path = temp_path
                break
        
        # Load images
        image = Image.open(img_path).convert('RGB')
        mask = Image.open(mask_path)
        
        # Resize
        image = transforms.functional.resize(image, (self.img_size, self.img_size), antialias=True)
        mask = transforms.functional.resize(mask, (self.img_size, self.img_size), 
                                           interpolation=transforms.InterpolationMode.NEAREST)
        
        if self.augment:
            image, mask = self._fast_augment(image, mask)
        
        # Convert to tensors
        image = transforms.functional.to_tensor(image)
        image = self.normalize(image)
        
        # Create fresh tensor for mask (avoid storage issues)
        mask_array = np.array(mask, dtype=np.int64)
        mask = torch.tensor(mask_array, dtype=torch.long)
        
        return image, mask

# ============================================================================
# LIGHTWEIGHT ASPP
# ============================================================================

class LightASPP(nn.Module):
    """Faster ASPP with fewer branches"""
    def __init__(self, in_channels, out_channels):
        super().__init__()
        
        self.conv1 = self._make_conv(in_channels, out_channels, 1, 0, 1)
        self.conv6 = self._make_conv(in_channels, out_channels, 3, 6, 6)
        self.conv12 = self._make_conv(in_channels, out_channels, 3, 12, 12)
        
        self.global_pool = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_channels, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )
        
        self.attention = FastCBAM(out_channels * 4, reduction=8)
        
        self.project = nn.Sequential(
            nn.Conv2d(out_channels * 4, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Dropout2d(0.15)
        )
    
    def _make_conv(self, in_ch, out_ch, k, pad, dil):
        return nn.Sequential(
            nn.Conv2d(in_ch, out_ch, k, padding=pad, dilation=dil, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True)
        )
    
    def forward(self, x):
        size = x.shape[-2:]
        
        f1 = self.conv1(x)
        f6 = self.conv6(x)
        f12 = self.conv12(x)
        fg = nn.functional.interpolate(self.global_pool(x), size=size, 
                                       mode='bilinear', align_corners=False)
        
        x = torch.cat([f1, f6, f12, fg], dim=1)
        x = self.attention(x)
        return self.project(x)

# ============================================================================
# SPEED-OPTIMIZED DeepLab
# ============================================================================

class FastDeepLab(nn.Module):
    """Streamlined DeepLab with strategic attention placement"""
    def __init__(self, num_classes=104, pretrained=True, dropout=0.15):
        super().__init__()
        
        # Use new weights API
        weights = ResNet50_Weights.IMAGENET1K_V2 if pretrained else None
        backbone = resnet50(weights=weights)
        
        self.layer0 = nn.Sequential(backbone.conv1, backbone.bn1, 
                                   backbone.relu, backbone.maxpool)
        self.layer1 = backbone.layer1
        self.layer2 = backbone.layer2
        self.layer3 = backbone.layer3
        self.layer4 = backbone.layer4
        
        # Strategic attention: only where it matters most
        self.att3 = FastCBAM(1024, reduction=8)  # After layer3
        self.att4 = EfficientSelfAttention(2048, reduction=8, downsample=2)  # After layer4
        
        self.aspp = LightASPP(2048, 256)
        
        # Simpler decoder
        self.decoder = nn.Sequential(
            nn.Conv2d(256, 256, 3, padding=1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            FastCBAM(256, reduction=8),
            nn.Dropout2d(dropout),
            nn.Conv2d(256, num_classes, 1)
        )
    
    def forward(self, x):
        input_size = x.shape[-2:]
        
        x = self.layer0(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.att3(self.layer3(x))
        x = self.att4(self.layer4(x))
        x = self.aspp(x)
        x = self.decoder(x)
        
        return {'out': nn.functional.interpolate(x, size=input_size, 
                                                 mode='bilinear', align_corners=False)}

# ============================================================================
# OPTIMIZED Training
# ============================================================================

def get_param_groups(model, backbone_lr, head_lr):
    backbone_params = []
    head_params = []
    
    for name, param in model.named_parameters():
        if 'layer' in name and any(f'layer{i}' in name for i in range(5)):
            backbone_params.append(param)
        else:
            head_params.append(param)
    
    return [
        {'params': backbone_params, 'lr': backbone_lr, 'weight_decay': 1e-4},
        {'params': head_params, 'lr': head_lr, 'weight_decay': 1e-4}
    ]


@torch.no_grad()
def compute_loss_wrapper(model, images, masks, criterion):
    """Wrapper to avoid torch.compile issues"""
    return criterion(model(images)['out'], masks)


def calculate_metrics_fast(pred, target, num_classes=104):
    """Faster metric calculation"""
    valid = (target != 255)
    pred = pred[valid]
    target = target[valid]
    
    if pred.size == 0:  # Use .size for numpy arrays
        return 0.0, 0.0
    
    acc = (pred == target).mean()
    
    # Vectorized IoU calculation
    ious = []
    for c in range(num_classes):
        pred_c = (pred == c)
        target_c = (target == c)
        inter = (pred_c & target_c).sum()
        union = (pred_c | target_c).sum()
        if union > 0:
            ious.append(inter / union)
    
    return acc, np.mean(ious) if ious else 0.0


def train_epoch(model, loader, criterion, optimizer, device, epoch, scaler):
    model.train()
    total_loss = total_miou = 0
    
    pbar = tqdm(loader, desc=f"Epoch {epoch}")
    for images, masks in pbar:
        images = images.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)
        
        optimizer.zero_grad(set_to_none=True)
        
        # Use newer API if available
        try:
            with torch.amp.autocast('cuda'):
                outputs = model(images)
                loss = criterion(outputs['out'], masks)
        except:
            with torch.cuda.amp.autocast():
                outputs = model(images)
                loss = criterion(outputs['out'], masks)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        # Calculate metrics less frequently for speed
        if random.random() < 0.3:  # 30% of batches
            with torch.no_grad():
                preds = outputs['out'].argmax(1)
                _, miou = calculate_metrics_fast(preds.cpu().numpy().ravel(), 
                                                masks.cpu().numpy().ravel())
                total_miou += miou
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.3f}'})
    
    return total_loss / len(loader), total_miou / max(1, len(loader) * 0.3)


@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()
    total_loss = total_miou = 0
    
    for images, masks in tqdm(loader, desc="Validating"):
        images = images.to(device, non_blocking=True)
        masks = masks.to(device, non_blocking=True)
        
        outputs = model(images)
        loss = criterion(outputs['out'], masks)
        preds = outputs['out'].argmax(1)
        
        _, miou = calculate_metrics_fast(preds.cpu().numpy().ravel(), 
                                        masks.cpu().numpy().ravel())
        
        total_loss += loss.item()
        total_miou += miou
    
    return total_loss / len(loader), total_miou / len(loader)

# ============================================================================
# MAIN
# ============================================================================

def train_fast(
    root_dir, 
    num_epochs=25,
    batch_size=24,          # Larger batch
    backbone_lr=2e-4,       # Higher LR for speed
    head_lr=2e-3,
    img_size=320,           # Smaller for speed
    warmup_epochs=2,
    dropout=0.15
):
    set_seed(42)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    print("="*60)
    print("SPEED-OPTIMIZED TRAINING")
    print("="*60)
    print(f"Image: {img_size}x{img_size} | Batch: {batch_size}")
    print(f"Custom attention: Fast implementations")
    print(f"Optimizations: cudnn.benchmark, torch.compile, cached paths")
    print("="*60 + "\n")
    
    # Datasets with caching
    train_ds = FastFoodSeg103Dataset(root_dir, 'train', img_size, augment=True)
    val_ds = FastFoodSeg103Dataset(root_dir, 'test', img_size, augment=False)
    
    train_loader = DataLoader(train_ds, batch_size, shuffle=True, 
                             num_workers=4, pin_memory=True, 
                             persistent_workers=True, prefetch_factor=2)
    val_loader = DataLoader(val_ds, batch_size*2, shuffle=False,
                           num_workers=4, pin_memory=True,
                           persistent_workers=True, prefetch_factor=2)
    
    model = FastDeepLab(104, pretrained=True, dropout=dropout).to(device)
    
    param_groups = get_param_groups(model, backbone_lr, head_lr)
    
    # Use standard AdamW (fused might not be available)
    try:
        optimizer = optim.AdamW(param_groups, fused=True)
    except:
        optimizer = optim.AdamW(param_groups)
    
    scheduler = optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=[backbone_lr*2, head_lr*2],
        epochs=num_epochs, 
        steps_per_epoch=len(train_loader),
        pct_start=0.1
    )
    
    criterion = nn.CrossEntropyLoss(ignore_index=255, label_smoothing=0.08)
    
    # Use new GradScaler API
    try:
        scaler = torch.amp.GradScaler('cuda')
    except:
        scaler = torch.cuda.amp.GradScaler()
    
    best_miou = 0
    patience = 0
    
    for epoch in range(1, num_epochs + 1):
        train_loss, train_miou = train_epoch(model, train_loader, criterion, 
                                            optimizer, device, epoch, scaler)
        val_loss, val_miou = validate(model, val_loader, criterion, device)
        
        scheduler.step()
        
        print(f"\nEpoch {epoch}: Train mIoU: {train_miou:.3f} | Val mIoU: {val_miou:.3f}")
        
        if val_miou > best_miou:
            best_miou = val_miou
            patience = 0
            torch.save({'model': model.state_dict(), 'miou': best_miou}, 
                      'best_fast_model.pth')
            print(f"✓ Saved! Best: {best_miou:.3f}")
        else:
            patience += 1
            if patience >= 8:
                print("Early stopping")
                break
    
    print(f"\n{'='*60}\nBest mIoU: {best_miou:.3f}\n{'='*60}")
    return model

if __name__ == "__main__":
    model = train_fast(
        root_dir="/kaggle/input/foodseg103",
        num_epochs=25,
        batch_size=24,
        backbone_lr=2e-4,
        head_lr=2e-3,
        img_size=320,
        warmup_epochs=2,
        dropout=0.15
    )